# Experiments IV
In this notebook we try the ML supervised classification approch on our use case Emotion-Detection

In [18]:
#Packages 
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import normalize
import tensorflow as tf

### Data

In [2]:
#Importing data
data = pd.read_excel ( 'D:/Users/chatbot_emotions_detection/Datasets/emtions_detection_datasets/02_final_data.xlsx' )
data.drop (columns = ['Unnamed: 0'], inplace = True)
print ('number of rows', data.shape[0])


#[s, lov, lo, j] to be deleted
idx_to_delete = data[data['emotion'].isin (['s', 'lov', 'lo', 'j'])].index
data_01 = data.drop (index= idx_to_delete)

#Doing the emotional mapping, emotion to their original form
emotion_mapping = {'sadness':'sad', 'Sad': 'sad',
                    'joy': 'happy', 'happiness': 'happy', 'fun': 'happy', 'Laughing': 'happy', 'Happy': 'happy', 'love': 'happy', 'enthusiasm': 'happy', 'relief': 'happy',
                    'none': 'neutral', 'Neutral': 'neutral', 'boredom': 'neutral', 'empty': 'neutral', 'sympathy': 'neutral', 'Wondering': 'neutral',
                    'Angry': 'angry', 'anger': 'angry', 'hate': 'angry',
                    'Surprised': 'surprised', 'surprise': 'surprised',
                    'worry': 'fear', 'fear': 'fear'}
#Remaining emotions : ['sad', 'happy', 'surprised', 'fear', 'angry', neutral]
data_01['emotion'] = data_01['emotion'].replace (emotion_mapping)

#Number of rows per emotion
px.bar (data_01['emotion'].value_counts(), width = 500, title = 'Number of rows per emotion')

number of rows 448643


In [3]:
#Numb of rows for each language
data_01['language'].value_counts()

language
eng          416809
egy           10065
arb-trans      8000
arb            6369
fr             5000
alg-latin      2400
Name: count, dtype: int64

### Testing ML models 

In [7]:
from transformers import XLMRobertaTokenizer, TFXLMRobertaModel, TFXLMRobertaForSequenceClassification

d:\Users\chatbot_emotions_detection\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



First we need to tokenize and extract embeddings from data to feed it to our ML models.

In [4]:
#THIS PREPROCESSING IS DESIGNED FOR ARABIC DATASETS 
#Spliting data
data = data_01 [data_01['language'].isin (["arb", "egy"])]
data = data.reset_index (drop= True)
data = data.drop ( columns = ['language'] )
#Drop null values and columns with neutral as emotion
data.dropna (inplace = True)
#drop duplicates values of column text
data = data.drop_duplicates ( subset = "text" )
#delet neutral columns
#idx_to_delete = data[data['langauge'] == "neutral"]


In [5]:
data

,text,emotion
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,neutral
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,angry
2,كتنا نيله ف حظنا الهباب xD,sad
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,happy
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,neutral
...,...,...
16424,اديش شكلك بايخ وأنت زعيمك مغير رأيه بموضوع داف...,angry
16426,قرف وزهق وملل ومشاكل وكله تعيس ومحدش عادطايق ح...,angry
16429,من الاصوات انا خذني و انا بزعل و انا حتى الزعل...,sad
16430,@imazizz ايش الحلو فيه دام بالاخير نهايه محزنه,sad


In [8]:
#Extracting model and tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = TFXLMRobertaModel.from_pretrained(model_name)

#
model.summary ()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "tfxlm_roberta_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFXLMRobertaMainL  multiple                  278043648 
 ayer)                                                           
                                                                 
Total params: 278043648 (1.04 GB)
Trainable params: 278043648 (1.04 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
#Tokenizing data
inputs = tokenizer ( data ['text'].to_list(), padding = True, truncation = True, return_tensors = "tf" )

#Let's encode the emotion column [emotion <--> label]
emotion_to_num = {"happy": 0, "sad": 1, "angry": 2, "fear": 3, "surprised": 4, "neutral": 5}
labels = [emotion_to_num[emotion] for emotion in data['emotion']]


# Convert input data and labels to TensorFlow tensors
'''train_inputs_tf = {'input_ids': tf.convert_to_tensor(train_inputs['input_ids']),
                   'attention_mask': tf.convert_to_tensor(train_inputs['attention_mask'])}
test_inputs_tf = {'input_ids': tf.convert_to_tensor(test_inputs['input_ids']),
                  'attention_mask': tf.convert_to_tensor(test_inputs['attention_mask'])}
train_labels_tf = tf.convert_to_tensor(train_labels)
test_labels_tf = tf.convert_to_tensor(test_labels)'''

"train_inputs_tf = {'input_ids': tf.convert_to_tensor(train_inputs['input_ids']),\n                   'attention_mask': tf.convert_to_tensor(train_inputs['attention_mask'])}\ntest_inputs_tf = {'input_ids': tf.convert_to_tensor(test_inputs['input_ids']),\n                  'attention_mask': tf.convert_to_tensor(test_inputs['attention_mask'])}\ntrain_labels_tf = tf.convert_to_tensor(train_labels)\ntest_labels_tf = tf.convert_to_tensor(test_labels)"

In [10]:
#Train test split
train_inputs, test_inputs, train_labels, test_labels =  train_test_split (inputs ['input_ids'].numpy (), labels, test_size = 0.07 )
print ( "train inputs size:", train_inputs.shape )
print ( "train labels size:", len (train_labels) )
print ( "train inputs size:", test_inputs.shape )
print ( "test labels size:", len (test_labels) )

train inputs size: (13344, 109)
train labels size: 13344
train inputs size: (1005, 109)
test labels size: 1005


In [14]:
#Tokens
inputs

{'input_ids': <tf.Tensor: shape=(14349, 109), dtype=int32, numpy=
array([[     0,  76804,    179, ...,      1,      1,      1],
       [     0,      6, 104498, ...,      1,      1,      1],
       [     0,   4316,  23579, ...,      1,      1,      1],
       ...,
       [     0,    230,   3484, ...,      1,      1,      1],
       [     0,   1374,   1352, ...,      1,      1,      1],
       [     0,  12011,  92480, ...,      1,      1,      1]])>, 'attention_mask': <tf.Tensor: shape=(14349, 109), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}

In [12]:
#Train tokens
train_inputs

array([[     0,   6067,  14386, ...,      1,      1,      1],
       [     0,  76804,    179, ...,      1,      1,      1],
       [     0,  17725,   4714, ...,      1,      1,      1],
       ...,
       [     0,     28,     12, ...,      1,      1,      1],
       [     0,   3039,      6, ...,      1,      1,      1],
       [     0,  43228, 100259, ...,      1,      1,      1]])

In [ ]:
"""#Embedding data per batch 
train_outputs = {}
for i in range (0, len (train_dataset), 1000):
    train_inputs = tokenizer ( train_dataset ['text'].to_list()[i:i+1000], padding = True, truncation = True, return_tensors = "tf" )
    train_outputs [f'outputs_{i}'] = model (**train_inputs)
    print ( "done for i =", i )
list_out = []
for key in train_outputs.keys ():
    if key != "outputs_13000":
        list_out.append ( train_outputs[key].last_hidden_state )
#list_out
# Concatenate the list of tensors along the first axis to get our embeddings
embeddings = tf.concat(list_out, axis=1)"""

In [ ]:
#Getting Embeddings
inputs = tokenizer ( data ['text'].to_list()[:5000], padding = True, truncation = True, return_tensors = "tf" )
outputs = model (**inputs)
embeddings = outputs.last_hidden_state.numpy ()

In [14]:
# Reshape the embeddings to (2000, 80*768)
embeddings = embeddings.reshape(embeddings.shape[0], -1)
#embeddings.shape 
embeddings 

array([[ 0.0768415 ,  0.07517537,  0.05109024, ..., -0.15667143,
         0.02465443,  0.02300712],
       [ 0.11614204,  0.07896394,  0.04477318, ..., -0.11905558,
         0.0422257 , -0.00140056],
       [ 0.0793714 ,  0.09689906,  0.07324526, ..., -0.15699881,
         0.01640409,  0.0528123 ],
       ...,
       [ 0.07332662,  0.09518625,  0.04800884, ..., -0.12764594,
         0.03098051,  0.02600136],
       [ 0.10144281,  0.08294697,  0.04697794, ..., -0.13645525,
         0.02039052,  0.01315391],
       [ 0.11623553,  0.16021524,  0.0909683 , ..., -0.27770475,
         0.00106859,  0.11739847]], dtype=float32)

In [20]:
#Normalizing tokens 
train_inputs_n = normalize ( train_inputs)
test_inputs_n = normalize (test_inputs )
train_inputs_n

array([[0.00000000e+00, 1.27338306e-02, 3.01943114e-02, ...,
        2.09886774e-06, 2.09886774e-06, 2.09886774e-06],
       [0.00000000e+00, 2.26229213e-01, 5.27251564e-04, ...,
        2.94553946e-06, 2.94553946e-06, 2.94553946e-06],
       [0.00000000e+00, 4.39313423e-02, 1.16836303e-02, ...,
        2.47849604e-06, 2.47849604e-06, 2.47849604e-06],
       ...,
       [0.00000000e+00, 7.74655838e-05, 3.31995359e-05, ...,
        2.76662799e-06, 2.76662799e-06, 2.76662799e-06],
       [0.00000000e+00, 1.28585195e-02, 2.53870079e-05, ...,
        4.23116798e-06, 4.23116798e-06, 4.23116798e-06],
       [0.00000000e+00, 2.51532586e-01, 5.83381270e-01, ...,
        5.81874215e-06, 5.81874215e-06, 5.81874215e-06]])

Model SVM

In [21]:
#Initialize and train SVM Model
svm_model = SVC()
svm_model.fit (train_inputs_n, train_labels )

SVC()

In [22]:
#Results on training data
y_predicted =  svm_model.predict (train_inputs_n )

accuracy = accuracy_score ( train_labels, y_predicted )
F1_score = f1_score (train_labels, y_predicted, average= "macro")

print ( "train accuracy =", accuracy )
print ( "train f1 score = ", F1_score )

train accuracy = 0.4072242206235012
train f1 score =  0.2469858384843545


In [23]:
#Results on test data
y_predicted =  svm_model.predict (test_inputs_n )

accuracy = accuracy_score ( test_labels, y_predicted )
F1_score = f1_score (test_labels, y_predicted, average= "macro")

print ( "test accuracy =", accuracy )
print ( "test f1 score = ", F1_score )

test accuracy = 0.272636815920398
test f1 score =  0.159636350666658


original metrics : 

test accuracy = 0.27562189054726366

test f1 score =  0.14605177984240045

Model Gradient Boosting 

In [24]:
#Initialize and train Gradient Boosting
gradient_boosting = GradientBoostingClassifier (n_estimators = 200)
gradient_boosting.fit ( train_inputs_n, train_labels )

GradientBoostingClassifier(n_estimators=200)

In [18]:
#Results on training data
y_predicted =  gradient_boosting.predict (train_inputs )

accuracy = accuracy_score ( train_labels, y_predicted )
F1_score = f1_score (train_labels, y_predicted, average= "macro")

print ( "train accuracy =", accuracy )
print ( "train f1 score = ", F1_score )

train accuracy = 0.5900779376498801
train f1 score =  0.5443816807730646


In [19]:
#Results on training data
y_predicted =  gradient_boosting.predict (test_inputs )

accuracy = accuracy_score ( test_labels, y_predicted )
F1_score = f1_score (test_labels, y_predicted, average= "macro")

print ( "test accuracy =", accuracy )
print ( "test f1 score = ", F1_score )

test accuracy = 0.39502487562189054
test f1 score =  0.33136793788926017


Model SVM embeddings

In [16]:
svm_model = SVC()
svm_model.fit (embeddings, train_labels[0:2000] )

SVC()

In [17]:
#Results on training data
y_predicted = svm_model.predict (embeddings )

accuracy = accuracy_score ( train_labels[0:2000], y_predicted )
F1_score = f1_score (train_labels[0:2000], y_predicted, average= "macro")

print ( "train accuracy =", accuracy )
print ( "train f1 score = ", F1_score )

train accuracy = 0.259
train f1 score =  0.0685729414879534
